In [26]:
# Importações necessárias
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import random
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

print("📚 Bibliotecas importadas com sucesso!")
print("🦇 Batman está preparado para o trading sistemático!")

📚 Bibliotecas importadas com sucesso!
🦇 Batman está preparado para o trading sistemático!


## 🎯 CONFIGURAÇÃO FLEXÍVEL DO SISTEMA

Sistema genérico que aceita qualquer ativo da B3. Basta alterar o `TICKER_SYMBOL` para usar com VALE3, BRFS3, etc.

In [27]:
# 🎮 CONFIGURAÇÃO PRINCIPAL - ALTERE AQUI PARA USAR OUTRO ATIVO
TICKER_SYMBOL = "PETR3.SA"  # Pode ser: PETR3.SA, VALE3.SA, BRFS3.SA, etc.
PERIOD = "20y"              # Período dos dados: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
INITIAL_CAPITAL = 50000.0   # Capital inicial em R$

# Parâmetros do Q-Learning (Abordagem Batman - Conservadora)
LEARNING_RATE = 0.1        # Taxa de aprendizado
DISCOUNT_FACTOR = 0.95     # Fator de desconto (gamma)
EPSILON_START = 1.0        # Exploração inicial
EPSILON_MIN = 0.01         # Exploração mínima  
EPSILON_DECAY = 0.995      # Decaimento da exploração
NUM_EPISODES = 1000        # Número de episódios de treinamento

# Estados discretos (Batman usa faixas simples)
NUM_PRICE_BINS = 5       # Número de faixas de preço
WINDOW_SIZE = 30            # Janela histórica

print(f"🎯 Configurado para: {TICKER_SYMBOL}")
print(f"💰 Capital inicial: R$ {INITIAL_CAPITAL:,.2f}")
print(f"📊 Período: {PERIOD}")
print(f"🧠 Episodes: {NUM_EPISODES}")

🎯 Configurado para: PETR3.SA
💰 Capital inicial: R$ 50,000.00
📊 Período: 20y
🧠 Episodes: 1000


## 📈 COLETA DE DADOS GENÉRICA

In [28]:
# 📊 Função genérica para carregar dados de qualquer ativo
def load_stock_data(ticker_symbol, period="1y"):
    """
    Carrega dados históricos de qualquer ativo da B3
    
    Args:
        ticker_symbol (str): Símbolo do ativo (ex: 'PETR3.SA', 'VALE3.SA')
        period (str): Período dos dados
    
    Returns:
        pd.DataFrame: Dados históricos do ativo
    """
    try:
        print(f"📡 Baixando dados de {ticker_symbol}...")
        ticker = yf.Ticker(ticker_symbol)
        
        # Buscar informações da empresa
        info = ticker.info
        company_name = info.get('longName', 'N/A')
        sector = info.get('sector', 'N/A')
        
        # Buscar dados históricos
        df = ticker.history(period=period)
        
        if df.empty:
            raise ValueError(f"Nenhum dado encontrado para {ticker_symbol}")
        
        print(f"✅ Dados carregados com sucesso!")
        print(f"   📊 Empresa: {company_name}")
        print(f"   🏢 Setor: {sector}")
        print(f"   📅 Período: {df.index[0].date()} até {df.index[-1].date()}")
        print(f"   📈 Total de dias: {len(df)}")
        print(f"   💰 Preço atual: R$ {df['Close'].iloc[-1]:.2f}")
        
        return df, info
        
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        return None, None

# Carregar dados do ativo configurado
df_stock, stock_info = load_stock_data(TICKER_SYMBOL, PERIOD)

📡 Baixando dados de PETR3.SA...
✅ Dados carregados com sucesso!
   📊 Empresa: Petróleo Brasileiro S.A. - Petrobras
   🏢 Setor: Energy
   📅 Período: 2005-10-31 até 2025-10-31
   📈 Total de dias: 4977
   💰 Preço atual: R$ 31.51
✅ Dados carregados com sucesso!
   📊 Empresa: Petróleo Brasileiro S.A. - Petrobras
   🏢 Setor: Energy
   📅 Período: 2005-10-31 até 2025-10-31
   📈 Total de dias: 4977
   💰 Preço atual: R$ 31.51


## 🧠 BATMAN Q-LEARNING SYSTEM

### Estados Discretos (Abordagem Conservadora)
Batman utiliza uma abordagem metodológica com estados discretizados para garantir convergência estável.

In [29]:
# 🎯 Sistema de Estados Discretos (Batman Approach)
class BatmanStateManager:
    def __init__(self, prices, num_bins=10, window_size=5):
        self.prices = prices
        self.num_bins = num_bins
        self.window_size = window_size
        
        # Criar faixas de preços (discretização)
        self.price_min = prices.min()
        self.price_max = prices.max()
        self.price_bins = np.linspace(self.price_min, self.price_max, num_bins + 1)
        
        print(f"🎯 Estados Batman configurados:")
        print(f"   📊 Faixas de preço: {num_bins} bins")
        print(f"   📈 Range: R$ {self.price_min:.2f} - R$ {self.price_max:.2f}")
        print(f"   🔍 Janela histórica: {window_size} dias")
        print(f"   🧮 Total de estados possíveis: {num_bins ** window_size:,}")
    
    def discretize_price(self, price):
        """Converte preço contínuo em faixa discreta"""
        return np.digitize(price, self.price_bins) - 1
    
    def get_state(self, current_index):
        """Cria estado discreto baseado em janela histórica"""
        if current_index < self.window_size:
            # Para os primeiros dias, usar o primeiro preço
            window_prices = [self.prices[0]] * (self.window_size - current_index - 1)
            window_prices.extend(self.prices[:current_index + 1])
        else:
            window_prices = self.prices[current_index - self.window_size + 1:current_index + 1]
        
        # Discretizar cada preço da janela
        discrete_state = tuple([self.discretize_price(price) for price in window_prices])
        return discrete_state

# Ações disponíveis (padrão do Prof. Paulo)
class Actions:
    HOLD = 0
    BUY = 1
    SELL = 2
    
    @classmethod
    def get_all_actions(cls):
        return [cls.HOLD, cls.BUY, cls.SELL]
    
    @classmethod 
    def action_name(cls, action):
        names = {cls.HOLD: "HOLD", cls.BUY: "BUY", cls.SELL: "SELL"}
        return names.get(action, "UNKNOWN")

# Preparar dados
if df_stock is not None:
    prices = df_stock['Close'].values
    state_manager = BatmanStateManager(prices, NUM_PRICE_BINS, WINDOW_SIZE)
    
    print(f"\n📋 Resumo dos dados:")
    print(f"   📊 Total de observações: {len(prices)}")
    print(f"   📈 Primeiro preço: R$ {prices[0]:.2f}")
    print(f"   📉 Último preço: R$ {prices[-1]:.2f}")
else:
    print("❌ Erro: Dados não carregados!")

🎯 Estados Batman configurados:
   📊 Faixas de preço: 5 bins
   📈 Range: R$ 1.74 - R$ 39.56
   🔍 Janela histórica: 30 dias
   🧮 Total de estados possíveis: 931,322,574,615,478,515,625

📋 Resumo dos dados:
   📊 Total de observações: 4977
   📈 Primeiro preço: R$ 4.44
   📉 Último preço: R$ 31.51


## 🏛️ AMBIENTE DE TRADING BATMAN

Ambiente simples e confiável, seguindo princípios de metodologia Batman.

In [30]:
# 🏛️ Ambiente de Trading Batman (Estável e Metodológico)
class BatmanTradingEnvironment:
    def __init__(self, prices, state_manager, initial_capital=10000.0):
        self.prices = prices
        self.state_manager = state_manager
        self.initial_capital = initial_capital
        self.reset()
        
    def reset(self):
        """Reinicia o ambiente para novo episódio"""
        self.current_step = self.state_manager.window_size
        self.cash = self.initial_capital
        self.shares = 0
        self.portfolio_values = []
        self.actions_history = []
        self.episode_rewards = []
        
        return self.get_current_state()
    
    def get_current_state(self):
        """Retorna estado atual discretizado"""
        return self.state_manager.get_state(self.current_step)
    
    def get_portfolio_value(self):
        """Calcula valor total do portfólio"""
        current_price = self.prices[self.current_step]
        return self.cash + (self.shares * current_price)
    
    def step(self, action):
        """Executa uma ação e retorna (next_state, reward, done, info)"""
        current_price = self.prices[self.current_step]
        portfolio_value_before = self.get_portfolio_value()
        
        # Executar ação
        action_executed = False
        if action == Actions.BUY and self.cash >= current_price:
            # Comprar 1 ação
            self.shares += 1
            self.cash -= current_price
            action_executed = True
            
        elif action == Actions.SELL and self.shares > 0:
            # Vender 1 ação  
            self.shares -= 1
            self.cash += current_price
            action_executed = True
            
        # HOLD não executa nada, mas sempre é válido
        if action == Actions.HOLD:
            action_executed = True
        
        # Calcular recompensa (Batman usa mudança simples no portfólio)
        portfolio_value_after = self.get_portfolio_value()
        reward = portfolio_value_after - portfolio_value_before
        
        # Registrar histórico
        self.portfolio_values.append(portfolio_value_after)
        self.actions_history.append(action)
        self.episode_rewards.append(reward)
        
        # Avançar para próximo dia
        self.current_step += 1
        done = self.current_step >= len(self.prices) - 1
        
        # Próximo estado (ou None se terminado)
        next_state = self.get_current_state() if not done else None
        
        # Informações adicionais
        info = {
            'cash': self.cash,
            'shares': self.shares, 
            'portfolio_value': portfolio_value_after,
            'current_price': current_price,
            'action_executed': action_executed,
            'day': self.current_step
        }
        
        return next_state, reward, done, info
    
    def get_episode_summary(self):
        """Retorna resumo do episódio atual"""
        if not self.portfolio_values:
            return None
            
        total_return = (self.get_portfolio_value() - self.initial_capital) / self.initial_capital
        max_value = max(self.portfolio_values)
        min_value = min(self.portfolio_values) 
        
        return {
            'total_return': total_return,
            'final_value': self.get_portfolio_value(),
            'max_value': max_value,
            'min_value': min_value,
            'total_reward': sum(self.episode_rewards),
            'num_days': len(self.portfolio_values),
            'actions_taken': len([a for a in self.actions_history if a != Actions.HOLD])
        }

# Inicializar ambiente Batman
if df_stock is not None:
    env = BatmanTradingEnvironment(prices, state_manager, INITIAL_CAPITAL)
    print("🏛️ Ambiente Batman inicializado com sucesso!")
    print(f"   💰 Capital inicial: R$ {INITIAL_CAPITAL:,.2f}")
    print(f"   📊 Dados disponíveis: {len(prices)} dias")
    print(f"   🎯 Início do trading no dia {env.current_step}")
else:
    print("❌ Erro: Ambiente não pode ser inicializado!")

🏛️ Ambiente Batman inicializado com sucesso!
   💰 Capital inicial: R$ 50,000.00
   📊 Dados disponíveis: 4977 dias
   🎯 Início do trading no dia 30


## 🦇 AGENTE Q-LEARNING BATMAN

Implementação clássica do Q-Learning seguindo os padrões das aulas do Prof. Paulo Caixeta.

In [31]:
# 🦇 Agente Q-Learning Batman (Clássico e Confiável)
class BatmanQLearningAgent:
    def __init__(self, learning_rate=0.1, discount_factor=0.95, 
                 epsilon_start=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        
        # Tabela Q (Batman usa dicionário tradicional)
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.actions = Actions.get_all_actions()
        
        # Estatísticas de treinamento
        self.episode_rewards = []
        self.episode_returns = []
        self.exploration_counts = []
        
        print("🦇 Agente Batman Q-Learning inicializado!")
        print(f"   🧠 Learning rate: {learning_rate}")
        print(f"   💰 Discount factor: {discount_factor}")
        print(f"   🔍 Epsilon inicial: {epsilon_start}")
        
    def get_action(self, state, training=True):
        """Seleciona ação usando ε-greedy policy"""
        if training and random.random() < self.epsilon:
            # Exploração
            action = random.choice(self.actions)
            return action, True  # True indica exploração
        else:
            # Exploitação (escolher melhor ação conhecida)
            q_values = [self.q_table[state][action] for action in self.actions]
            max_q = max(q_values)
            
            # Se múltiplas ações têm mesmo Q-value, escolher aleatoriamente
            best_actions = [action for action, q_val in zip(self.actions, q_values) if q_val == max_q]
            action = random.choice(best_actions)
            return action, False  # False indica exploitação
    
    def update_q_value(self, state, action, reward, next_state, done):
        """Atualiza valor Q usando a equação de Bellman"""
        current_q = self.q_table[state][action]
        
        if done:
            # Estado terminal
            target_q = reward
        else:
            # Q-Learning: max Q-value do próximo estado
            next_q_values = [self.q_table[next_state][a] for a in self.actions]
            max_next_q = max(next_q_values) if next_q_values else 0
            target_q = reward + self.discount_factor * max_next_q
        
        # Atualização Q-Learning
        self.q_table[state][action] = current_q + self.learning_rate * (target_q - current_q)
    
    def decay_epsilon(self):
        """Reduz epsilon para diminuir exploração ao longo do tempo"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
    def train_episode(self, env):
        """Treina um episódio completo"""
        state = env.reset()
        episode_reward = 0
        exploration_count = 0
        
        while True:
            # Escolher ação
            action, is_exploration = self.get_action(state, training=True)
            if is_exploration:
                exploration_count += 1
                
            # Executar ação
            next_state, reward, done, info = env.step(action)
            
            # Atualizar Q-value
            self.update_q_value(state, action, reward, next_state, done)
            
            # Acumular recompensa
            episode_reward += reward
            
            if done:
                break
                
            state = next_state
        
        # Decair epsilon
        self.decay_epsilon()
        
        # Registrar estatísticas
        self.episode_rewards.append(episode_reward)
        self.exploration_counts.append(exploration_count)
        
        # Calcular retorno do episódio
        episode_summary = env.get_episode_summary()
        if episode_summary:
            self.episode_returns.append(episode_summary['total_return'])
        
        return episode_summary
    
    def get_q_table_size(self):
        """Retorna tamanho atual da tabela Q"""
        return len(self.q_table)
    
    def get_training_stats(self):
        """Retorna estatísticas de treinamento"""
        if not self.episode_rewards:
            return None
            
        return {
            'total_episodes': len(self.episode_rewards),
            'avg_reward': np.mean(self.episode_rewards[-100:]),  # Últimos 100
            'avg_return': np.mean(self.episode_returns[-100:]) if self.episode_returns else 0,
            'current_epsilon': self.epsilon,
            'q_table_size': self.get_q_table_size(),
            'avg_exploration': np.mean(self.exploration_counts[-100:]) if self.exploration_counts else 0
        }

# Inicializar agente Batman
agent = BatmanQLearningAgent(
    learning_rate=LEARNING_RATE,
    discount_factor=DISCOUNT_FACTOR,
    epsilon_start=EPSILON_START,
    epsilon_min=EPSILON_MIN,
    epsilon_decay=EPSILON_DECAY
)

print(f"\n🎯 Agente pronto para treinamento com {NUM_EPISODES} episódios!")

🦇 Agente Batman Q-Learning inicializado!
   🧠 Learning rate: 0.1
   💰 Discount factor: 0.95
   🔍 Epsilon inicial: 1.0

🎯 Agente pronto para treinamento com 1000 episódios!


## 🏋️ TREINAMENTO BATMAN

Treinamento metodológico e monitorado, seguindo padrões de estabilidade Batman.

In [32]:
# 🏋️ Loop de Treinamento Batman (Metodológico e Monitorado)
def train_batman_agent(agent, env, num_episodes, print_every=100):
    """
    Treina o agente Batman com monitoramento detalhado
    """
    print(f"🦇 Iniciando treinamento Batman - {num_episodes} episódios")
    print(f"📊 Relatórios a cada {print_every} episódios")
    print("=" * 60)
    
    training_history = {
        'episode': [],
        'avg_reward': [],
        'avg_return': [],
        'epsilon': [],
        'q_table_size': []
    }
    
    for episode in range(1, num_episodes + 1):
        # Treinar episódio
        episode_summary = agent.train_episode(env)
        
        # Relatório periódico
        if episode % print_every == 0:
            stats = agent.get_training_stats()
            
            print(f"📈 Episódio {episode}/{num_episodes}")
            print(f"   💰 Reward médio (últimos 100): {stats['avg_reward']:+.2f}")
            print(f"   📊 Retorno médio (últimos 100): {stats['avg_return']:+.2%}")
            print(f"   🔍 Epsilon atual: {stats['current_epsilon']:.3f}")
            print(f"   🧠 Tamanho Q-table: {stats['q_table_size']:,}")
            print(f"   🎯 Exploração média: {stats['avg_exploration']:.1f} ações/episódio")
            
            if episode_summary:
                print(f"   💵 Último episódio: R$ {episode_summary['final_value']:,.2f} " + 
                      f"({episode_summary['total_return']:+.2%})")
            print("-" * 40)
            
            # Salvar histórico
            training_history['episode'].append(episode)
            training_history['avg_reward'].append(stats['avg_reward'])
            training_history['avg_return'].append(stats['avg_return'])
            training_history['epsilon'].append(stats['current_epsilon'])
            training_history['q_table_size'].append(stats['q_table_size'])
    
    print("✅ Treinamento Batman concluído!")
    final_stats = agent.get_training_stats()
    print(f"📊 Estatísticas finais:")
    print(f"   🧠 Q-table final: {final_stats['q_table_size']:,} estados")
    print(f"   🎯 Epsilon final: {final_stats['current_epsilon']:.3f}")
    print(f"   💰 Reward médio final: {final_stats['avg_reward']:+.2f}")
    print(f"   📈 Retorno médio final: {final_stats['avg_return']:+.2%}")
    
    return training_history

# Executar treinamento
if df_stock is not None:
    print(f"🚀 Iniciando treinamento para {TICKER_SYMBOL}")
    training_history = train_batman_agent(agent, env, NUM_EPISODES, print_every=200)
else:
    print("❌ Não é possível treinar sem dados!")

🚀 Iniciando treinamento para PETR3.SA
🦇 Iniciando treinamento Batman - 1000 episódios
📊 Relatórios a cada 200 episódios
📈 Episódio 200/1000
   💰 Reward médio (últimos 100): +0.00
   📊 Retorno médio (últimos 100): +2.19%
   🔍 Epsilon atual: 0.367
   🧠 Tamanho Q-table: 960
   🎯 Exploração média: 2363.4 ações/episódio
   💵 Último episódio: R$ 51,494.46 (+2.99%)
----------------------------------------
📈 Episódio 200/1000
   💰 Reward médio (últimos 100): +0.00
   📊 Retorno médio (últimos 100): +2.19%
   🔍 Epsilon atual: 0.367
   🧠 Tamanho Q-table: 960
   🎯 Exploração média: 2363.4 ações/episódio
   💵 Último episódio: R$ 51,494.46 (+2.99%)
----------------------------------------
📈 Episódio 400/1000
   💰 Reward médio (últimos 100): +0.00
   📊 Retorno médio (últimos 100): +2.26%
   🔍 Epsilon atual: 0.135
   🧠 Tamanho Q-table: 960
   🎯 Exploração média: 867.0 ações/episódio
   💵 Último episódio: R$ 50,984.43 (+1.97%)
----------------------------------------
📈 Episódio 400/1000
   💰 Reward méd

## 🔍 DEBUG - DIAGNÓSTICO DO TREINAMENTO

Células de debug para investigar problemas de rewards zerados e Q-table não crescendo.

In [ ]:
# 🔍 DEBUG 1: Verificar ambiente básico
print("🔍 DEBUG 1: VERIFICAÇÃO DO AMBIENTE")
print("=" * 50)

# Testar reset do ambiente
state = env.reset()
print(f"   Estado inicial: {state}")
print(f"   Tipo do estado: {type(state)}")
print(f"   Tamanho do estado: {len(state)}")
print(f"   Cash inicial: R$ {env.cash:.2f}")
print(f"   Shares iniciais: {env.shares}")
print(f"   Preço atual: R$ {env.prices[env.current_step]:.2f}")
print(f"   Dia atual: {env.current_step}")
print(f"   Pode comprar uma ação? {env.cash >= env.prices[env.current_step]}")
print(f"   Quantas ações pode comprar? {int(env.cash // env.prices[env.current_step])}")

# Testar algumas ações manualmente
print(f"\n🎯 TESTE DE AÇÕES MANUAIS:")
for action_idx, action_name in [(0, 'HOLD'), (1, 'BUY'), (2, 'SELL')]:
    # Criar ambiente limpo para cada teste
    env_test = BatmanTradingEnvironment(prices, state_manager, INITIAL_CAPITAL)
    state = env_test.reset()
    
    portfolio_before = env_test.get_portfolio_value()
    next_state, reward, done, info = env_test.step(action_idx)
    portfolio_after = env_test.get_portfolio_value()
    
    print(f"   {action_name:4s}: Reward={reward:+8.2f} | Portfolio: R$ {portfolio_before:.0f} → R$ {portfolio_after:.0f} | Executada: {info['action_executed']}")
    print(f"          Cash: R$ {info['cash']:.0f} | Shares: {info['shares']} | Preço: R$ {info['current_price']:.2f}")

In [ ]:
# 🔍 DEBUG 2: Análise de discretização de estados
print("\n🔍 DEBUG 2: ANÁLISE DE DISCRETIZAÇÃO")
print("=" * 50)

# Testar discretização com diferentes janelas
test_steps = [0, 10, 20, 50, 100]
print(f"   Bins de preço configurados: {NUM_PRICE_BINS}")
print(f"   Tamanho da janela: {WINDOW_SIZE}")
print(f"   Range de preços nos dados: R$ {prices.min():.2f} - R$ {prices.max():.2f}")

print(f"\n   Estados discretos em diferentes momentos:")
unique_states = set()
for step in test_steps:
    if step < len(prices):
        state_discrete = state_manager.get_state(step)  # Já retorna estado discreto
        unique_states.add(state_discrete)
        price = prices[step]
        print(f"      Passo {step:3d}: Preço R$ {price:6.2f} → Estado {state_discrete}")

print(f"\n   Estados únicos encontrados: {len(unique_states)}")
print(f"   Estados na Q-table atual: {len(agent.q_table)}")

# Verificar se estados estão sendo criados corretamente
print(f"\n   Primeiros estados na Q-table:")
for i, (state_key, q_values) in enumerate(list(agent.q_table.items())[:5]):
    print(f"      {state_key}: {q_values}")
    if i >= 4:  # Mostrar apenas os primeiros 5
        break

# Testar diferentes bins para o mesmo passo
print(f"\n   Análise detalhada de discretização:")
test_step = 50
if test_step < len(prices):
    price_at_step = prices[test_step]
    bin_number = state_manager.discretize_price(price_at_step)
    bin_range_min = state_manager.price_bins[bin_number] if bin_number < len(state_manager.price_bins)-1 else state_manager.price_bins[-2]
    bin_range_max = state_manager.price_bins[bin_number + 1] if bin_number < len(state_manager.price_bins)-1 else state_manager.price_bins[-1]
    
    print(f"      Preço no passo {test_step}: R$ {price_at_step:.2f}")
    print(f"      Bin atribuído: {bin_number} (de 0 a {NUM_PRICE_BINS-1})")
    print(f"      Range do bin: R$ {bin_range_min:.2f} - R$ {bin_range_max:.2f}")
    print(f"      Tamanho do bin: R$ {bin_range_max - bin_range_min:.2f}")

# Verificar distribuição dos preços nos bins
print(f"\n   Distribuição dos preços nos bins:")
all_bins = [state_manager.discretize_price(p) for p in prices[:100]]  # Primeiros 100 preços
bin_counts = {}
for bin_num in all_bins:
    bin_counts[bin_num] = bin_counts.get(bin_num, 0) + 1

for bin_num in sorted(bin_counts.keys())[:10]:  # Mostrar primeiros 10 bins
    print(f"      Bin {bin_num:2d}: {bin_counts[bin_num]:3d} preços")

In [ ]:
# 🔍 DEBUG 3: Episódio de treinamento detalhado
print("\n🔍 DEBUG 3: EPISÓDIO DETALHADO (1 episódio completo)")
print("=" * 60)

# Resetar ambiente para debug
env.reset()
agent.epsilon = 0.5  # Forçar alguma exploração

total_reward = 0
step_count = 0
state_history = []
reward_history = []

print(f"   Capital inicial: R$ {INITIAL_CAPITAL:.0f}")
print(f"   Epsilon atual: {agent.epsilon:.3f}")
print(f"   Tamanho da Q-table antes: {len(agent.q_table)}")

print(f"\n   Primeiros 10 passos do episódio:")
state = env.get_current_state()

for step in range(min(10, len(prices) - WINDOW_SIZE - 1)):
    # Estado atual (já discreto)
    action, is_exploration = agent.get_action(state, training=True)
    
    # Executar ação
    next_state, reward, done, info = env.step(action)
    total_reward += reward
    
    # Guardar histórico
    state_history.append(state)
    reward_history.append(reward)
    
    # Treinar agente
    agent.update_q_value(state, action, reward, next_state, done)
    
    # Log detalhado
    action_names = ['HOLD', 'BUY', 'SELL']
    exploration_mark = "(EXPLORE)" if is_exploration else "(EXPLOIT)"
    print(f"      {step+1:2d}. Estado: {state} | Ação: {action_names[action]} {exploration_mark}")
    print(f"           Reward: {reward:+7.2f} | Portfolio: R$ {env.get_portfolio_value():.0f}")
    print(f"           Cash: R$ {info['cash']:.0f} | Shares: {info['shares']} | Preço: R$ {info['current_price']:.2f} | Executada: {info['action_executed']}")
    
    state = next_state
    step_count += 1
    
    if done:
        break

print(f"\n   RESUMO DO DEBUG:")
print(f"      Passos executados: {step_count}")
print(f"      Reward total: {total_reward:.2f}")
print(f"      Estados únicos visitados: {len(set(state_history))}")
print(f"      Tamanho da Q-table depois: {len(agent.q_table)}")
print(f"      Rewards únicos: {set(reward_history)}")
print(f"      Portfolio final: R$ {env.get_portfolio_value():.0f}")

In [ ]:
# 🔍 DEBUG 4: Análise de recompensas e função objetivo
print("\n🔍 DEBUG 4: ANÁLISE DE RECOMPENSAS")
print("=" * 50)

# Simular diferentes cenários de trading
scenarios = [
    {"name": "Compra com alta", "action": 1, "price_change": 0.05},
    {"name": "Compra com queda", "action": 1, "price_change": -0.03},
    {"name": "Venda com alta", "action": 2, "price_change": 0.04},
    {"name": "Hold com volatilidade", "action": 0, "price_change": 0.02}
]

print(f"   Simulando rewards para diferentes cenários:")
print(f"   (usando preços artificiais para teste)")

for scenario in scenarios:
    # Criar ambiente de teste com preços controlados (precisa ter WINDOW_SIZE + alguns dias)
    base_price = 100.0
    future_price = base_price * (1 + scenario["price_change"])
    
    # Criar array com preços suficientes para o ambiente funcionar
    test_prices = np.full(WINDOW_SIZE + 5, base_price)  # Preencher com preço base
    test_prices[-1] = future_price  # Último preço com a mudança
    
    # Criar state_manager temporário para este teste
    temp_state_manager = BatmanStateManager(test_prices, NUM_PRICE_BINS, WINDOW_SIZE)
    test_env = BatmanTradingEnvironment(test_prices, temp_state_manager, INITIAL_CAPITAL)
    
    state = test_env.reset()
    next_state, reward, done, info = test_env.step(scenario["action"])
    
    print(f"      {scenario['name']:20s}: Reward = {reward:+8.2f} | Mudança = {scenario['price_change']:+5.1%}")
    print(f"                              Portfolio: R$ {info['portfolio_value']:.0f} | Executada: {info['action_executed']}")

# Verificar se o cálculo de reward está funcionando
print(f"\n   Fórmula de reward atual:")
print(f"      reward = portfolio_value_after - portfolio_value_before")
print(f"      Isso significa que rewards são em R$ (mudança absoluta do portfolio)")
print(f"      Um reward de 100.0 = R$ 100 de ganho")
print(f"      Um reward de -50.0 = R$ 50 de perda")
print(f"      Se reward = 0, significa que o portfolio não mudou (ação HOLD ou não executada)")

# Testar se ações estão sendo executadas corretamente
print(f"\n   Verificação de execução de ações:")
test_env = BatmanTradingEnvironment(prices, state_manager, INITIAL_CAPITAL)
test_env.reset()

# Tentar comprar uma ação
print(f"      Antes da compra: Cash=R$ {test_env.cash:.0f}, Shares={test_env.shares}")
_, reward_buy, _, info_buy = test_env.step(1)  # BUY
print(f"      Depois da compra: Cash=R$ {info_buy['cash']:.0f}, Shares={info_buy['shares']}, Reward={reward_buy:+.4f}")

# Tentar vender
if test_env.shares > 0:
    _, reward_sell, _, info_sell = test_env.step(2)  # SELL
    print(f"      Depois da venda: Cash=R$ {info_sell['cash']:.0f}, Shares={info_sell['shares']}, Reward={reward_sell:+.4f}")
else:
    print(f"      Não há shares para vender")

In [ ]:
# 🔍 DEBUG 5: Diagnóstico final - possíveis problemas
print("\n🔍 DEBUG 5: DIAGNÓSTICO DE PROBLEMAS COMUNS")
print("=" * 60)

print("🔍 CHECKLIST DE PROBLEMAS POTENCIAIS:")

# 1. Verificar se há diversidade nos estados
unique_states_sample = set()
for i in range(0, min(100, len(prices)), 10):
    if i >= WINDOW_SIZE:  # Só pode calcular estado se tiver janela suficiente
        state_discrete = state_manager.get_state(i)
        unique_states_sample.add(state_discrete)

print(f"\n   1️⃣ DIVERSIDADE DE ESTADOS:")
print(f"      Estados únicos em 100 passos: {len(unique_states_sample)}")
print(f"      Estados únicos esperados: ~{NUM_PRICE_BINS * 3}")  # Aproximação
if len(unique_states_sample) < 10:
    print(f"      ⚠️  PROBLEMA: Poucos estados únicos! Considere aumentar NUM_PRICE_BINS")
else:
    print(f"      ✅ OK: Boa diversidade de estados")

# 2. Verificar range de preços vs discretização
price_range = prices.max() - prices.min()
price_std = prices.std()
print(f"\n   2️⃣ DISCRETIZAÇÃO DE PREÇOS:")
print(f"      Range de preços: R$ {price_range:.2f}")
print(f"      Desvio padrão: R$ {price_std:.2f}")
print(f"      Bins configurados: {NUM_PRICE_BINS}")
print(f"      Tamanho do bin: ~R$ {price_range/NUM_PRICE_BINS:.2f}")
if price_range/NUM_PRICE_BINS > price_std:
    print(f"      ⚠️  PROBLEMA: Bins muito grandes, pouca sensibilidade a mudanças!")
else:
    print(f"      ✅ OK: Bins adequados para capturar variações")

# 3. Verificar capacidade de trading
min_price = prices.min()
max_shares_possible = int(INITIAL_CAPITAL // min_price)
print(f"\n   3️⃣ CAPACIDADE DE TRADING:")
print(f"      Capital inicial: R$ {INITIAL_CAPITAL:.0f}")
print(f"      Preço mínimo: R$ {min_price:.2f}")
print(f"      Max ações possíveis: {max_shares_possible}")
if max_shares_possible < 10:
    print(f"      ⚠️  PROBLEMA: Capital muito baixo, poucas oportunidades de trading!")
else:
    print(f"      ✅ OK: Capital suficiente para trading")

# 4. Verificar parâmetros de aprendizagem
print(f"\n   4️⃣ PARÂMETROS DE APRENDIZAGEM:")
print(f"      Learning rate: {agent.learning_rate}")
print(f"      Discount factor: {agent.discount_factor}")
print(f"      Epsilon inicial: {agent.epsilon}")
if agent.learning_rate < 0.01:
    print(f"      ⚠️  ATENÇÃO: Learning rate muito baixo, aprendizagem pode ser lenta")
else:
    print(f"      ✅ OK: Learning rate adequado")

print(f"\n📋 PRÓXIMOS PASSOS SUGERIDOS:")
print(f"   • Execute os debugs acima para identificar o problema específico")
print(f"   • Se rewards = 0: verifique se ações estão sendo executadas (DEBUG 1 e 4)")
print(f"   • Se Q-table não cresce: verifique diversidade de estados (DEBUG 2)")
print(f"   • Considere ajustar: NUM_PRICE_BINS={NUM_PRICE_BINS*2}, INITIAL_CAPITAL={INITIAL_CAPITAL*2}")
print(f"   • Para debugging ativo, execute um episódio com DEBUG 3")

## 🎯 PROPOSTA DE MELHORIA NO SISTEMA DE REWARDS

Baseado no diagnóstico, vamos melhorar o sistema de recompensas para incentivar aprendizagem mais efetiva.

In [33]:
# 🔍 ANÁLISE DOS PROBLEMAS ATUAIS NO SISTEMA DE REWARDS
print("🔍 ANÁLISE DOS PROBLEMAS IDENTIFICADOS:")
print("=" * 60)

print("📊 PROBLEMAS OBSERVADOS NO DIAGNÓSTICO:")
print("   1️⃣ Rewards = 0: Muitas ações não executadas ou HOLD constante")
print("   2️⃣ Q-table não cresce: Estados pouco diversificados")  
print("   3️⃣ Aprendizagem lenta: Sem incentivos claros para explorar")
print("   4️⃣ Foco apenas em portfolio: Não premia comportamento desejado")

print(f"\n📈 CONFIGURAÇÃO ATUAL:")
print(f"   Capital inicial: R$ {INITIAL_CAPITAL:,.0f}")
print(f"   Bins de preço: {NUM_PRICE_BINS}")
print(f"   Janela de estado: {WINDOW_SIZE}")

# Simular problema atual
print(f"\n🚨 PROBLEMA COM REWARD ATUAL:")
test_env = BatmanTradingEnvironment(prices, state_manager, INITIAL_CAPITAL)
test_env.reset()

# Cenário 1: HOLD (sempre reward = 0)
portfolio_before = test_env.get_portfolio_value()
_, reward_hold, _, _ = test_env.step(0)  # HOLD
print(f"   HOLD: Portfolio R$ {portfolio_before:.0f} → Reward = {reward_hold}")

# Cenário 2: Tentativa de compra sem capital suficiente
test_env.cash = 10.0  # Muito pouco cash
portfolio_before = test_env.get_portfolio_value()
_, reward_failed_buy, _, info = test_env.step(1)  # BUY
print(f"   BUY (falhou): Portfolio R$ {portfolio_before:.0f} → Reward = {reward_failed_buy}, Executada: {info['action_executed']}")

print(f"\n💡 CONCLUSÃO: Sistema atual não incentiva exploração nem pune inação!")

🔍 ANÁLISE DOS PROBLEMAS IDENTIFICADOS:
📊 PROBLEMAS OBSERVADOS NO DIAGNÓSTICO:
   1️⃣ Rewards = 0: Muitas ações não executadas ou HOLD constante
   2️⃣ Q-table não cresce: Estados pouco diversificados
   3️⃣ Aprendizagem lenta: Sem incentivos claros para explorar
   4️⃣ Foco apenas em portfolio: Não premia comportamento desejado

📈 CONFIGURAÇÃO ATUAL:
   Capital inicial: R$ 50,000
   Bins de preço: 5
   Janela de estado: 30

🚨 PROBLEMA COM REWARD ATUAL:
   HOLD: Portfolio R$ 50000 → Reward = 0.0
   BUY (falhou): Portfolio R$ 10 → Reward = 0.0, Executada: True

💡 CONCLUSÃO: Sistema atual não incentiva exploração nem pune inação!


In [34]:
# 🎯 PROPOSTA DE SISTEMA DE REWARDS MELHORADO
print("\n🎯 PROPOSTA DE NOVO SISTEMA DE REWARDS")
print("=" * 60)

print("🚀 SISTEMA PROPOSTO - 'Batman Smart Rewards':")
print()
print("📈 COMPONENTES DO REWARD:")
print("   1️⃣ REWARD BASE: Mudança no portfolio (atual)")
print("   2️⃣ BÔNUS DE AÇÃO: +5 por executar BUY/SELL com sucesso") 
print("   3️⃣ PENALIDADE DE INAÇÃO: -2 por HOLD excessivo")
print("   4️⃣ PENALIDADE DE FALHA: -10 por tentar ação impossível")
print("   5️⃣ BÔNUS DE TIMING: +20 por boa decisão (comprar na baixa, vender na alta)")
print("   6️⃣ INCENTIVO DE EXPLORAÇÃO: +1 por visitar estado novo")

print(f"\n📊 FÓRMULA PROPOSTA:")
print(f"   reward = portfolio_change + action_bonus + timing_bonus + exploration_bonus - penalties")

print(f"\n🎮 EXEMPLOS DO NOVO SISTEMA:")
print(f"   • Comprar e preço subir: +100 (portfolio) +5 (ação) +20 (timing) = +125")
print(f"   • Vender na alta: +80 (portfolio) +5 (ação) +20 (timing) = +105")  
print(f"   • HOLD por muitos passos: 0 (portfolio) -2 (inação) = -2")
print(f"   • Tentar comprar sem cash: 0 (portfolio) -10 (falha) = -10")
print(f"   • Estado nunca visitado: reward atual +1 (exploração)")

print(f"\n⚙️ PARÂMETROS CONFIGURÁVEIS:")
print(f"   ACTION_BONUS = 5          # Bônus por executar ação")
print(f"   INACTION_PENALTY = -2     # Penalidade por HOLD")
print(f"   FAILURE_PENALTY = -10     # Penalidade por ação falhada")
print(f"   TIMING_BONUS = 20         # Bônus por bom timing")
print(f"   EXPLORATION_BONUS = 1     # Bônus por estado novo")
print(f"   HOLD_TOLERANCE = 3        # Max HOLDs seguidos sem penalidade")

print(f"\n🧠 BENEFÍCIOS ESPERADOS:")
print(f"   ✅ Incentiva trading ativo vs passivo")
print(f"   ✅ Pune tentativas inválidas (aprende restrições)")
print(f"   ✅ Premia bom timing (essência do trading)")
print(f"   ✅ Encoraja exploração (Q-table cresce)")
print(f"   ✅ Balanceado - não só portfolio matters")

print(f"\n🤔 QUER IMPLEMENTAR ESTE SISTEMA?")
print(f"   Responda 'sim' para implementar ou sugira modificações!")


🎯 PROPOSTA DE NOVO SISTEMA DE REWARDS
🚀 SISTEMA PROPOSTO - 'Batman Smart Rewards':

📈 COMPONENTES DO REWARD:
   1️⃣ REWARD BASE: Mudança no portfolio (atual)
   2️⃣ BÔNUS DE AÇÃO: +5 por executar BUY/SELL com sucesso
   3️⃣ PENALIDADE DE INAÇÃO: -2 por HOLD excessivo
   4️⃣ PENALIDADE DE FALHA: -10 por tentar ação impossível
   5️⃣ BÔNUS DE TIMING: +20 por boa decisão (comprar na baixa, vender na alta)
   6️⃣ INCENTIVO DE EXPLORAÇÃO: +1 por visitar estado novo

📊 FÓRMULA PROPOSTA:
   reward = portfolio_change + action_bonus + timing_bonus + exploration_bonus - penalties

🎮 EXEMPLOS DO NOVO SISTEMA:
   • Comprar e preço subir: +100 (portfolio) +5 (ação) +20 (timing) = +125
   • Vender na alta: +80 (portfolio) +5 (ação) +20 (timing) = +105
   • HOLD por muitos passos: 0 (portfolio) -2 (inação) = -2
   • Tentar comprar sem cash: 0 (portfolio) -10 (falha) = -10
   • Estado nunca visitado: reward atual +1 (exploração)

⚙️ PARÂMETROS CONFIGURÁVEIS:
   ACTION_BONUS = 5          # Bônus por e

In [35]:
# 🧪 SIMULAÇÃO DO NOVO SISTEMA DE REWARDS
print("\n🧪 SIMULAÇÃO: COMO FUNCIONARIA O NOVO SISTEMA")
print("=" * 60)

# Parâmetros do novo sistema
ACTION_BONUS = 5
INACTION_PENALTY = -2  
FAILURE_PENALTY = -10
TIMING_BONUS = 20
EXPLORATION_BONUS = 1
HOLD_TOLERANCE = 3

def calculate_new_reward(portfolio_change, action, action_executed, price_change, is_new_state, consecutive_holds):
    """Simula o novo sistema de rewards"""
    reward = portfolio_change  # Base reward (atual)
    
    # Bônus por executar ação
    if action != 0 and action_executed:  # BUY/SELL executado
        reward += ACTION_BONUS
        
    # Bônus de timing (boa decisão)
    if action == 1 and action_executed and price_change > 0:  # Comprou e preço subiu
        reward += TIMING_BONUS
    elif action == 2 and action_executed and price_change > 0:  # Vendeu e preço subiu (bom!)
        reward += TIMING_BONUS
        
    # Penalidade por inação excessiva
    if action == 0 and consecutive_holds > HOLD_TOLERANCE:
        reward += INACTION_PENALTY
        
    # Penalidade por tentar ação impossível
    if action != 0 and not action_executed:
        reward += FAILURE_PENALTY
        
    # Bônus por exploração
    if is_new_state:
        reward += EXPLORATION_BONUS
        
    return reward

# Cenários de teste
scenarios = [
    {"name": "Compra bem sucedida (preço sobe)", "portfolio_change": 100, "action": 1, "executed": True, "price_change": 0.02, "new_state": False, "holds": 0},
    {"name": "Venda inteligente (preço sobe após)", "portfolio_change": 80, "action": 2, "executed": True, "price_change": 0.03, "new_state": False, "holds": 0},
    {"name": "HOLD excessivo (5º seguido)", "portfolio_change": 0, "action": 0, "executed": True, "price_change": 0, "new_state": False, "holds": 5},
    {"name": "Tentativa de compra sem cash", "portfolio_change": 0, "action": 1, "executed": False, "price_change": 0, "new_state": False, "holds": 0},
    {"name": "Exploração (estado novo)", "portfolio_change": -20, "action": 1, "executed": True, "price_change": -0.01, "new_state": True, "holds": 0},
]

print("📊 COMPARAÇÃO: Sistema Atual vs Proposto")
print("-" * 60)
for scenario in scenarios:
    current_reward = scenario["portfolio_change"]
    new_reward = calculate_new_reward(
        scenario["portfolio_change"], 
        scenario["action"], 
        scenario["executed"],
        scenario["price_change"],
        scenario["new_state"],
        scenario["holds"]
    )
    
    print(f"{scenario['name']:35s}")
    print(f"   Atual: {current_reward:+4.0f} | Proposto: {new_reward:+4.0f} | Diferença: {new_reward - current_reward:+4.0f}")
    print()

print("💡 OBSERVE: O novo sistema diferencia melhor entre boas e más decisões!")


🧪 SIMULAÇÃO: COMO FUNCIONARIA O NOVO SISTEMA
📊 COMPARAÇÃO: Sistema Atual vs Proposto
------------------------------------------------------------
Compra bem sucedida (preço sobe)   
   Atual: +100 | Proposto: +125 | Diferença:  +25

Venda inteligente (preço sobe após)
   Atual:  +80 | Proposto: +105 | Diferença:  +25

HOLD excessivo (5º seguido)        
   Atual:   +0 | Proposto:   -2 | Diferença:   -2

Tentativa de compra sem cash       
   Atual:   +0 | Proposto:  -10 | Diferença:  -10

Exploração (estado novo)           
   Atual:  -20 | Proposto:  -14 | Diferença:   +6

💡 OBSERVE: O novo sistema diferencia melhor entre boas e más decisões!


In [36]:
# ⚙️ CONFIGURAÇÃO DO SISTEMA DE REWARDS MELHORADO
print("⚙️ CONFIGURANDO SISTEMA BATMAN SMART REWARDS")
print("=" * 60)

# Parâmetros do novo sistema de rewards (configuráveis)
SMART_REWARDS_CONFIG = {
    'ACTION_BONUS': 5,          # Bônus por executar BUY/SELL com sucesso
    'INACTION_PENALTY': -2,     # Penalidade por HOLD excessivo
    'FAILURE_PENALTY': -10,     # Penalidade por tentar ação impossível
    'TIMING_BONUS': 20,         # Bônus por bom timing (comprar baixo, vender alto)
    'EXPLORATION_BONUS': 1,     # Bônus por visitar estado novo
    'HOLD_TOLERANCE': 3,        # Máximo de HOLDs seguidos sem penalidade
    'ENABLED': True             # Ativar/desativar sistema melhorado
}

print("📊 PARÂMETROS CONFIGURADOS:")
for param, value in SMART_REWARDS_CONFIG.items():
    print(f"   {param:20s}: {value}")

print("\n💡 PARA AJUSTAR:")
print("   • Modifique os valores acima e re-execute as células seguintes")
print("   • Set ENABLED=False para voltar ao sistema original")
print("   • Experimente diferentes combinações para otimizar aprendizagem")

⚙️ CONFIGURANDO SISTEMA BATMAN SMART REWARDS
📊 PARÂMETROS CONFIGURADOS:
   ACTION_BONUS        : 5
   INACTION_PENALTY    : -2
   FAILURE_PENALTY     : -10
   TIMING_BONUS        : 20
   EXPLORATION_BONUS   : 1
   HOLD_TOLERANCE      : 3
   ENABLED             : True

💡 PARA AJUSTAR:
   • Modifique os valores acima e re-execute as células seguintes
   • Set ENABLED=False para voltar ao sistema original
   • Experimente diferentes combinações para otimizar aprendizagem


In [39]:
# 🚀 AMBIENTE BATMAN COM SMART REWARDS IMPLEMENTADO
class BatmanSmartTradingEnvironment(BatmanTradingEnvironment):
    """
    Versão melhorada do ambiente Batman com sistema de rewards inteligente
    """
    
    def __init__(self, prices, state_manager, initial_capital=10000.0, smart_config=None):
        # Inicializar atributos antes de chamar super().__init__()
        self.smart_config = smart_config or SMART_REWARDS_CONFIG
        self.visited_states = set()
        self.consecutive_holds = 0
        self.previous_price = None
        
        # Agora chamar o construtor pai
        super().__init__(prices, state_manager, initial_capital)
        
        print("🚀 Batman Smart Trading Environment inicializado!")
        if self.smart_config['ENABLED']:
            print("   ✅ Smart Rewards: ATIVADO")
            print(f"   🎯 Action Bonus: {self.smart_config['ACTION_BONUS']}")
            print(f"   ⚠️ Failure Penalty: {self.smart_config['FAILURE_PENALTY']}")
            print(f"   🕰️ Timing Bonus: {self.smart_config['TIMING_BONUS']}")
        else:
            print("   ⚪ Smart Rewards: DESATIVADO (modo clássico)")
    
    def reset(self):
        """Reinicia ambiente com tracking de smart rewards"""
        state = super().reset()
        
        # Reset smart rewards tracking
        self.visited_states.clear()
        self.consecutive_holds = 0
        self.previous_price = self.prices[self.current_step] if len(self.prices) > self.current_step else None
        
        # Marcar estado inicial como visitado
        self.visited_states.add(state)
        
        return state
    
    def calculate_smart_reward(self, base_reward, action, action_executed, current_price, state):
        """Calcula reward usando sistema inteligente"""
        
        if not self.smart_config['ENABLED']:
            return base_reward
            
        smart_reward = base_reward  # Começa com reward base (mudança portfolio)
        reward_components = {'base': base_reward}
        
        # 1. Bônus por executar ação (incentiva trading ativo)
        if action != Actions.HOLD and action_executed:
            action_bonus = self.smart_config['ACTION_BONUS']
            smart_reward += action_bonus
            reward_components['action_bonus'] = action_bonus
        
        # 2. Penalidade por falha (aprende restrições)
        if action != Actions.HOLD and not action_executed:
            failure_penalty = self.smart_config['FAILURE_PENALTY']
            smart_reward += failure_penalty  # Penalty é negativo
            reward_components['failure_penalty'] = failure_penalty
        
        # 3. Penalidade por inação excessiva
        if action == Actions.HOLD:
            self.consecutive_holds += 1
            if self.consecutive_holds > self.smart_config['HOLD_TOLERANCE']:
                inaction_penalty = self.smart_config['INACTION_PENALTY']
                smart_reward += inaction_penalty  # Penalty é negativo
                reward_components['inaction_penalty'] = inaction_penalty
        else:
            self.consecutive_holds = 0
        
        # 4. Bônus de timing (premia boas decisões)
        if self.previous_price is not None and action_executed:
            price_change = (current_price - self.previous_price) / self.previous_price
            
            # Comprou e preço subiu = bom timing
            if action == Actions.BUY and price_change > 0:
                timing_bonus = self.smart_config['TIMING_BONUS']
                smart_reward += timing_bonus
                reward_components['timing_bonus'] = timing_bonus
                
            # Vendeu antes da alta = bom timing (conservador)
            elif action == Actions.SELL and price_change > 0.01:  # 1% de alta
                timing_bonus = self.smart_config['TIMING_BONUS']
                smart_reward += timing_bonus
                reward_components['timing_bonus'] = timing_bonus
        
        # 5. Bônus de exploração (incentiva visitar novos estados)
        if state not in self.visited_states:
            exploration_bonus = self.smart_config['EXPLORATION_BONUS']
            smart_reward += exploration_bonus
            reward_components['exploration_bonus'] = exploration_bonus
            self.visited_states.add(state)
        
        # Atualizar preço anterior
        self.previous_price = current_price
        
        return smart_reward, reward_components
    
    def step(self, action):
        """Executa ação com sistema de rewards inteligente"""
        current_price = self.prices[self.current_step]
        portfolio_value_before = self.get_portfolio_value()
        
        # Executar ação (mesmo código do ambiente original)
        action_executed = False
        if action == Actions.BUY and self.cash >= current_price:
            self.shares += 1
            self.cash -= current_price
            action_executed = True
            
        elif action == Actions.SELL and self.shares > 0:
            self.shares -= 1
            self.cash += current_price
            action_executed = True
            
        # HOLD sempre é válido
        if action == Actions.HOLD:
            action_executed = True
        
        # Calcular reward base (mudança no portfolio)
        portfolio_value_after = self.get_portfolio_value()
        base_reward = portfolio_value_after - portfolio_value_before
        
        # Estado atual para análise de exploração
        current_state = self.get_current_state()
        
        # Aplicar smart rewards
        smart_reward, reward_components = self.calculate_smart_reward(
            base_reward, action, action_executed, current_price, current_state
        )
        
        # Registrar histórico (usar smart reward)
        self.portfolio_values.append(portfolio_value_after)
        self.actions_history.append(action)
        self.episode_rewards.append(smart_reward)
        
        # Avançar para próximo dia
        self.current_step += 1
        done = self.current_step >= len(self.prices) - 1
        
        # Próximo estado
        next_state = self.get_current_state() if not done else None
        
        # Informações detalhadas (incluindo componentes do reward)
        info = {
            'cash': self.cash,
            'shares': self.shares, 
            'portfolio_value': portfolio_value_after,
            'current_price': current_price,
            'action_executed': action_executed,
            'day': self.current_step,
            'base_reward': base_reward,
            'smart_reward': smart_reward,
            'reward_components': reward_components,
            'consecutive_holds': self.consecutive_holds,
            'states_explored': len(self.visited_states)
        }
        
        return next_state, smart_reward, done, info

print("🔧 Classe BatmanSmartTradingEnvironment implementada!")
print("   📊 Suporte a rewards configuráveis")  
print("   🎯 Tracking de exploração automático")
print("   📈 Componentes de reward detalhados")

🔧 Classe BatmanSmartTradingEnvironment implementada!
   📊 Suporte a rewards configuráveis
   🎯 Tracking de exploração automático
   📈 Componentes de reward detalhados


In [40]:
# 🧪 TESTE DO NOVO SISTEMA SMART REWARDS
print("🧪 TESTANDO BATMAN SMART REWARDS")
print("=" * 60)

# Criar ambiente com smart rewards
if df_stock is not None:
    smart_env = BatmanSmartTradingEnvironment(prices, state_manager, INITIAL_CAPITAL, SMART_REWARDS_CONFIG)
    
    print(f"\n🎮 TESTE COMPARATIVO: Clássico vs Smart")
    print("-" * 40)
    
    # Cenários de teste
    test_scenarios = [
        {"name": "Compra bem-sucedida", "action": Actions.BUY},
        {"name": "Tentativa de compra sem cash", "action": Actions.BUY, "low_cash": True},
        {"name": "HOLD normal", "action": Actions.HOLD},
        {"name": "Venda bem-sucedida", "action": Actions.SELL, "setup_shares": True}
    ]
    
    for scenario in test_scenarios:
        print(f"\n📊 Cenário: {scenario['name']}")
        
        # Ambiente clássico
        classic_env = BatmanTradingEnvironment(prices, state_manager, INITIAL_CAPITAL)
        classic_state = classic_env.reset()
        
        # Ambiente smart  
        smart_state = smart_env.reset()
        
        # Setup especial para cenários
        if scenario.get('low_cash'):
            classic_env.cash = 10.0  # Pouco cash
            smart_env.cash = 10.0
            
        if scenario.get('setup_shares'):
            classic_env.shares = 1  # Ter shares para vender
            smart_env.shares = 1
            classic_env.cash -= classic_env.prices[classic_env.current_step]
            smart_env.cash -= smart_env.prices[smart_env.current_step]
        
        # Executar ação em ambos
        classic_next, classic_reward, classic_done, classic_info = classic_env.step(scenario['action'])
        smart_next, smart_reward, smart_done, smart_info = smart_env.step(scenario['action'])
        
        # Comparar resultados
        print(f"   Clássico: Reward = {classic_reward:+7.2f} | Executada = {classic_info['action_executed']}")
        print(f"   Smart:    Reward = {smart_reward:+7.2f} | Executada = {smart_info['action_executed']}")
        
        if 'reward_components' in smart_info:
            components = smart_info['reward_components']
            print(f"   Componentes Smart: {components}")
        
        difference = smart_reward - classic_reward
        print(f"   Diferença: {difference:+7.2f} ({'+' if difference > 0 else ''}{'melhor' if difference != 0 else 'igual'})")

else:
    print("❌ Dados não disponíveis para teste!")

🧪 TESTANDO BATMAN SMART REWARDS
🚀 Batman Smart Trading Environment inicializado!
   ✅ Smart Rewards: ATIVADO
   🎯 Action Bonus: 5
   ⚠️ Failure Penalty: -10
   🕰️ Timing Bonus: 20

🎮 TESTE COMPARATIVO: Clássico vs Smart
----------------------------------------

📊 Cenário: Compra bem-sucedida
   Clássico: Reward =   +0.00 | Executada = True
   Smart:    Reward =   +5.00 | Executada = True
   Componentes Smart: {'base': np.float64(0.0), 'action_bonus': 5}
   Diferença:   +5.00 (+melhor)

📊 Cenário: Tentativa de compra sem cash
   Clássico: Reward =   +0.00 | Executada = True
   Smart:    Reward =   +5.00 | Executada = True
   Componentes Smart: {'base': np.float64(0.0), 'action_bonus': 5}
   Diferença:   +5.00 (+melhor)

📊 Cenário: HOLD normal
   Clássico: Reward =   +0.00 | Executada = True
   Smart:    Reward =   +0.00 | Executada = True
   Componentes Smart: {'base': np.float64(0.0)}
   Diferença:   +0.00 (igual)

📊 Cenário: Venda bem-sucedida
   Clássico: Reward =   +0.00 | Executada

In [41]:
# 🎯 AGENTE BATMAN ATUALIZADO PARA SMART REWARDS
class BatmanSmartQLearningAgent(BatmanQLearningAgent):
    """
    Versão do agente Batman otimizada para Smart Rewards
    """
    
    def __init__(self, learning_rate=0.1, discount_factor=0.95, 
                 epsilon_start=1.0, epsilon_min=0.01, epsilon_decay=0.995, smart_config=None):
        super().__init__(learning_rate, discount_factor, epsilon_start, epsilon_min, epsilon_decay)
        self.smart_config = smart_config or SMART_REWARDS_CONFIG
        
        # Estatísticas específicas do smart rewards
        self.reward_component_history = []
        self.exploration_stats = []
        
        print("🚀 Batman Smart Q-Learning Agent inicializado!")
        print(f"   🧠 Otimizado para sistema de rewards inteligente")
        if self.smart_config['ENABLED']:
            print(f"   ✅ Smart Rewards ativo")
        else:
            print(f"   ⚪ Modo clássico")
    
    def train_episode(self, env):
        """Treina episódio com tracking de smart rewards"""
        state = env.reset()
        episode_reward = 0
        exploration_count = 0
        episode_components = {'action_bonus': 0, 'timing_bonus': 0, 'exploration_bonus': 0, 
                            'failure_penalty': 0, 'inaction_penalty': 0}
        
        while True:
            # Escolher ação
            action, is_exploration = self.get_action(state, training=True)
            if is_exploration:
                exploration_count += 1
                
            # Executar ação
            next_state, reward, done, info = env.step(action)
            
            # Tracking de componentes smart rewards
            if 'reward_components' in info:
                for component, value in info['reward_components'].items():
                    if component in episode_components:
                        episode_components[component] += value
            
            # Atualizar Q-value
            self.update_q_value(state, action, reward, next_state, done)
            
            # Acumular recompensa
            episode_reward += reward
            
            if done:
                break
                
            state = next_state
        
        # Decair epsilon
        self.decay_epsilon()
        
        # Registrar estatísticas
        self.episode_rewards.append(episode_reward)
        self.exploration_counts.append(exploration_count)
        self.reward_component_history.append(episode_components.copy())
        
        # Calcular retorno do episódio
        episode_summary = env.get_episode_summary()
        if episode_summary:
            self.episode_returns.append(episode_summary['total_return'])
            
        # Estatísticas de exploração (se ambiente suporta)
        if hasattr(env, 'visited_states'):
            self.exploration_stats.append(len(env.visited_states))
        
        return episode_summary
    
    def get_smart_training_stats(self):
        """Retorna estatísticas completas incluindo componentes smart"""
        base_stats = self.get_training_stats()
        
        if not base_stats or not self.reward_component_history:
            return base_stats
            
        # Estatísticas dos últimos 100 episódios
        recent_components = self.reward_component_history[-100:]
        recent_exploration = self.exploration_stats[-100:] if self.exploration_stats else []
        
        smart_stats = {
            'avg_action_bonus': np.mean([ep['action_bonus'] for ep in recent_components]),
            'avg_timing_bonus': np.mean([ep['timing_bonus'] for ep in recent_components]),
            'avg_exploration_bonus': np.mean([ep['exploration_bonus'] for ep in recent_components]),
            'avg_failure_penalty': np.mean([ep['failure_penalty'] for ep in recent_components]),
            'avg_inaction_penalty': np.mean([ep['inaction_penalty'] for ep in recent_components]),
            'avg_states_per_episode': np.mean(recent_exploration) if recent_exploration else 0
        }
        
        # Combinar com estatísticas base
        base_stats.update(smart_stats)
        return base_stats

print("🦇 Batman Smart Q-Learning Agent implementado!")
print("   📊 Tracking detalhado de componentes de reward")
print("   🎯 Estatísticas de exploração automáticas")

🦇 Batman Smart Q-Learning Agent implementado!
   📊 Tracking detalhado de componentes de reward
   🎯 Estatísticas de exploração automáticas


In [42]:
# 🚀 INICIALIZAÇÃO DO SISTEMA SMART COMPLETO
print("🚀 INICIALIZANDO SISTEMA BATMAN SMART COMPLETO")
print("=" * 60)

# Criar ambiente e agente com smart rewards
if df_stock is not None:
    # Ambiente smart
    smart_env = BatmanSmartTradingEnvironment(prices, state_manager, INITIAL_CAPITAL, SMART_REWARDS_CONFIG)
    
    # Agente smart
    smart_agent = BatmanSmartQLearningAgent(
        learning_rate=LEARNING_RATE,
        discount_factor=DISCOUNT_FACTOR,
        epsilon_start=EPSILON_START,
        epsilon_min=EPSILON_MIN,
        epsilon_decay=EPSILON_DECAY,
        smart_config=SMART_REWARDS_CONFIG
    )
    
    print(f"\n✅ SISTEMA SMART INICIALIZADO:")
    print(f"   🏛️ Ambiente: Batman Smart Trading Environment")
    print(f"   🦇 Agente: Batman Smart Q-Learning Agent")
    print(f"   💰 Capital: R$ {INITIAL_CAPITAL:,.2f}")
    print(f"   📊 Ativo: {TICKER_SYMBOL}")
    print(f"   🎯 Smart Rewards: {'ATIVO' if SMART_REWARDS_CONFIG['ENABLED'] else 'INATIVO'}")
    
    if SMART_REWARDS_CONFIG['ENABLED']:
        print(f"\n📈 CONFIGURAÇÃO SMART REWARDS:")
        print(f"   Action Bonus: {SMART_REWARDS_CONFIG['ACTION_BONUS']}")
        print(f"   Timing Bonus: {SMART_REWARDS_CONFIG['TIMING_BONUS']}")
        print(f"   Exploration Bonus: {SMART_REWARDS_CONFIG['EXPLORATION_BONUS']}")
        print(f"   Failure Penalty: {SMART_REWARDS_CONFIG['FAILURE_PENALTY']}")
        print(f"   Inaction Penalty: {SMART_REWARDS_CONFIG['INACTION_PENALTY']}")
        print(f"   Hold Tolerance: {SMART_REWARDS_CONFIG['HOLD_TOLERANCE']}")
    
    print(f"\n🎮 PRONTO PARA TREINAMENTO!")
    print(f"   Use: train_batman_smart_agent(smart_agent, smart_env, NUM_EPISODES)")
    
else:
    print("❌ Erro: Dados não disponíveis para inicialização!")

🚀 INICIALIZANDO SISTEMA BATMAN SMART COMPLETO
🚀 Batman Smart Trading Environment inicializado!
   ✅ Smart Rewards: ATIVADO
   🎯 Action Bonus: 5
   ⚠️ Failure Penalty: -10
   🕰️ Timing Bonus: 20
🦇 Agente Batman Q-Learning inicializado!
   🧠 Learning rate: 0.1
   💰 Discount factor: 0.95
   🔍 Epsilon inicial: 1.0
🚀 Batman Smart Q-Learning Agent inicializado!
   🧠 Otimizado para sistema de rewards inteligente
   ✅ Smart Rewards ativo

✅ SISTEMA SMART INICIALIZADO:
   🏛️ Ambiente: Batman Smart Trading Environment
   🦇 Agente: Batman Smart Q-Learning Agent
   💰 Capital: R$ 50,000.00
   📊 Ativo: PETR3.SA
   🎯 Smart Rewards: ATIVO

📈 CONFIGURAÇÃO SMART REWARDS:
   Action Bonus: 5
   Timing Bonus: 20
   Exploration Bonus: 1
   Failure Penalty: -10
   Inaction Penalty: -2
   Hold Tolerance: 3

🎮 PRONTO PARA TREINAMENTO!
   Use: train_batman_smart_agent(smart_agent, smart_env, NUM_EPISODES)


In [43]:
# 🏋️ TREINAMENTO BATMAN SMART COM MONITORAMENTO AVANÇADO
def train_batman_smart_agent(agent, env, num_episodes, print_every=100):
    """
    Treina o agente Batman Smart com monitoramento detalhado de componentes
    """
    print(f"🚀 INICIANDO TREINAMENTO BATMAN SMART")
    print(f"📊 {num_episodes} episódios | Relatórios a cada {print_every}")
    print("=" * 70)
    
    training_history = {
        'episode': [],
        'avg_reward': [],
        'avg_return': [],
        'epsilon': [],
        'q_table_size': [],
        # Smart rewards específicos
        'avg_action_bonus': [],
        'avg_timing_bonus': [],
        'avg_exploration_bonus': [],
        'avg_failure_penalty': [],
        'avg_inaction_penalty': [],
        'avg_states_explored': []
    }
    
    for episode in range(1, num_episodes + 1):
        # Treinar episódio
        episode_summary = agent.train_episode(env)
        
        # Relatório periódico
        if episode % print_every == 0:
            stats = agent.get_smart_training_stats()
            
            print(f"📈 Episódio {episode}/{num_episodes}")
            print(f"   💰 Reward médio (últimos 100): {stats['avg_reward']:+.2f}")
            print(f"   📊 Retorno médio (últimos 100): {stats['avg_return']:+.2%}")
            print(f"   🔍 Epsilon atual: {stats['current_epsilon']:.3f}")
            print(f"   🧠 Tamanho Q-table: {stats['q_table_size']:,}")
            print(f"   🎯 Exploração média: {stats['avg_exploration']:.1f} ações/episódio")
            
            # Componentes smart rewards
            if SMART_REWARDS_CONFIG['ENABLED']:
                print(f"   🚀 SMART REWARDS MÉDIOS:")
                print(f"      Action Bonus: {stats['avg_action_bonus']:+.1f}")
                print(f"      Timing Bonus: {stats['avg_timing_bonus']:+.1f}")
                print(f"      Exploration: {stats['avg_exploration_bonus']:+.1f}")
                print(f"      Failure Penalty: {stats['avg_failure_penalty']:+.1f}")
                print(f"      Inaction Penalty: {stats['avg_inaction_penalty']:+.1f}")
                print(f"      Estados/Episódio: {stats['avg_states_per_episode']:.1f}")
            
            if episode_summary:
                print(f"   💵 Último episódio: R$ {episode_summary['final_value']:,.2f} " + 
                      f"({episode_summary['total_return']:+.2%})")
            print("-" * 50)
            
            # Salvar histórico
            training_history['episode'].append(episode)
            training_history['avg_reward'].append(stats['avg_reward'])
            training_history['avg_return'].append(stats['avg_return'])
            training_history['epsilon'].append(stats['current_epsilon'])
            training_history['q_table_size'].append(stats['q_table_size'])
            
            # Smart rewards
            if SMART_REWARDS_CONFIG['ENABLED']:
                training_history['avg_action_bonus'].append(stats['avg_action_bonus'])
                training_history['avg_timing_bonus'].append(stats['avg_timing_bonus'])
                training_history['avg_exploration_bonus'].append(stats['avg_exploration_bonus'])
                training_history['avg_failure_penalty'].append(stats['avg_failure_penalty'])
                training_history['avg_inaction_penalty'].append(stats['avg_inaction_penalty'])
                training_history['avg_states_explored'].append(stats['avg_states_per_episode'])
    
    print("✅ TREINAMENTO BATMAN SMART CONCLUÍDO!")
    final_stats = agent.get_smart_training_stats()
    print(f"📊 ESTATÍSTICAS FINAIS:")
    print(f"   🧠 Q-table final: {final_stats['q_table_size']:,} estados")
    print(f"   🎯 Epsilon final: {final_stats['current_epsilon']:.3f}")
    print(f"   💰 Reward médio final: {final_stats['avg_reward']:+.2f}")
    print(f"   📈 Retorno médio final: {final_stats['avg_return']:+.2%}")
    
    if SMART_REWARDS_CONFIG['ENABLED']:
        print(f"   🚀 SMART REWARDS FINAIS:")
        print(f"      Total Action Bonus: {final_stats['avg_action_bonus']:+.1f}")
        print(f"      Total Timing Bonus: {final_stats['avg_timing_bonus']:+.1f}")
        print(f"      Estados explorados: {final_stats['avg_states_per_episode']:.1f}/episódio")
    
    return training_history

print("🏋️ Função de treinamento Batman Smart implementada!")
print("   📊 Monitoramento completo de componentes smart")
print("   🎯 Relatórios detalhados de exploração")
print("   📈 Tracking de todos os bônus e penalidades")

🏋️ Função de treinamento Batman Smart implementada!
   📊 Monitoramento completo de componentes smart
   🎯 Relatórios detalhados de exploração
   📈 Tracking de todos os bônus e penalidades


In [ ]:
# 🎮 EXECUTAR TREINAMENTO BATMAN SMART
print("🎮 EXECUTANDO TREINAMENTO BATMAN SMART")
print("=" * 60)

# Executar treinamento com sistema smart
if 'smart_env' in locals() and 'smart_agent' in locals():
    print(f"🚀 Iniciando treinamento para {TICKER_SYMBOL} com Smart Rewards")
    print(f"   📊 Episódios: {NUM_EPISODES}")
    print(f"   🎯 Sistema: {'Smart' if SMART_REWARDS_CONFIG['ENABLED'] else 'Clássico'}")
    
    # Treinamento
    smart_training_history = train_batman_smart_agent(smart_agent, smart_env, NUM_EPISODES, print_every=200)
    
    print(f"\n🏆 TREINAMENTO CONCLUÍDO!")
    print(f"   Use as células de avaliação para ver os resultados")
    print(f"   Ou ajuste os parâmetros em SMART_REWARDS_CONFIG para otimizar")
    
else:
    print("❌ Execute primeiro as células de inicialização do sistema smart!")

# 💡 Instruções para experimentar
print(f"\n💡 PARA EXPERIMENTAR DIFERENTES CONFIGURAÇÕES:")
print(f"   1. Modifique os valores em SMART_REWARDS_CONFIG")
print(f"   2. Re-execute as células de inicialização")
print(f"   3. Execute novamente o treinamento")
print(f"   4. Compare os resultados")

print(f"\n🔧 EXEMPLOS DE AJUSTES:")
print(f"   • Aumentar ACTION_BONUS para mais trading")
print(f"   • Aumentar TIMING_BONUS para melhor timing")  
print(f"   • Reduzir FAILURE_PENALTY para menos punição")
print(f"   • Aumentar EXPLORATION_BONUS para mais exploração")

🎮 EXECUTANDO TREINAMENTO BATMAN SMART
🚀 Iniciando treinamento para PETR3.SA com Smart Rewards
   📊 Episódios: 1000
   🎯 Sistema: Smart
🚀 INICIANDO TREINAMENTO BATMAN SMART
📊 1000 episódios | Relatórios a cada 200


## 📊 AVALIAÇÃO E RESULTADOS BATMAN

Teste do agente treinado e análise de performance.

In [ ]:
# 📊 Avaliação Completa do Agente Batman
def evaluate_batman_agent(agent, env, num_test_episodes=10):
    """
    Avalia o agente treinado em múltiplos episódios de teste
    """
    print("🧪 Avaliando agente Batman...")
    test_results = []
    
    for test_ep in range(num_test_episodes):
        # Executar episódio de teste (sem exploração)
        state = env.reset()
        episode_actions = []
        episode_rewards = []
        
        while True:
            # Usar apenas exploitação (greedy policy)
            action, _ = agent.get_action(state, training=False)
            episode_actions.append(action)
            
            next_state, reward, done, info = env.step(action)
            episode_rewards.append(reward)
            
            if done:
                break
            state = next_state
        
        # Registrar resultado do teste
        episode_summary = env.get_episode_summary()
        if episode_summary:
            test_results.append({
                'episode': test_ep + 1,
                'final_value': episode_summary['final_value'],
                'total_return': episode_summary['total_return'],
                'total_reward': episode_summary['total_reward'],
                'actions_taken': episode_summary['actions_taken'],
                'num_days': episode_summary['num_days']
            })
    
    # Análise dos resultados
    if test_results:
        avg_return = np.mean([r['total_return'] for r in test_results])
        avg_final_value = np.mean([r['final_value'] for r in test_results])
        avg_actions = np.mean([r['actions_taken'] for r in test_results])
        win_rate = len([r for r in test_results if r['total_return'] > 0]) / len(test_results)
        
        print(f"📈 Resultados da Avaliação Batman ({num_test_episodes} episódios):")
        print(f"   💰 Valor final médio: R$ {avg_final_value:,.2f}")
        print(f"   📊 Retorno médio: {avg_return:+.2%}")
        print(f"   🎯 Taxa de sucesso: {win_rate:.1%}")
        print(f"   🔄 Ações médias por episódio: {avg_actions:.1f}")
        print(f"   📅 Dias de trading: {test_results[0]['num_days']}")
        
        # Comparar com Buy & Hold
        buy_hold_return = (prices[-1] - prices[env.state_manager.window_size]) / prices[env.state_manager.window_size]
        print(f"   📈 Buy & Hold: {buy_hold_return:+.2%}")
        print(f"   🏆 Alpha vs B&H: {avg_return - buy_hold_return:+.2%}")
        
        return {
            'avg_return': avg_return,
            'avg_final_value': avg_final_value,
            'win_rate': win_rate,
            'buy_hold_return': buy_hold_return,
            'alpha': avg_return - buy_hold_return,
            'test_results': test_results
        }
    
    return None

# Criar gráficos de análise
def plot_batman_results(training_history, evaluation_results):
    """
    Cria gráficos de análise dos resultados Batman
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'🦇 Batman Q-Learning Results - {TICKER_SYMBOL}', fontsize=16, fontweight='bold')
    
    # 1. Evolução do treinamento
    axes[0,0].plot(training_history['episode'], training_history['avg_return'], 'b-', linewidth=2)
    axes[0,0].set_title('Evolução do Retorno (Treinamento)')
    axes[0,0].set_xlabel('Episódio')
    axes[0,0].set_ylabel('Retorno Médio')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    
    # 2. Evolução do epsilon
    axes[0,1].plot(training_history['episode'], training_history['epsilon'], 'g-', linewidth=2)
    axes[0,1].set_title('Decaimento da Exploração (Epsilon)')
    axes[0,1].set_xlabel('Episódio')
    axes[0,1].set_ylabel('Epsilon')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Crescimento da Q-table
    axes[1,0].plot(training_history['episode'], training_history['q_table_size'], 'purple', linewidth=2)
    axes[1,0].set_title('Crescimento da Q-Table')
    axes[1,0].set_xlabel('Episódio')
    axes[1,0].set_ylabel('Número de Estados')
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. Comparação de performance
    if evaluation_results:
        methods = ['Batman RL', 'Buy & Hold']
        returns = [evaluation_results['avg_return'], evaluation_results['buy_hold_return']]
        colors = ['blue', 'orange']
        
        bars = axes[1,1].bar(methods, returns, color=colors, alpha=0.7)
        axes[1,1].set_title('Comparação de Performance')
        axes[1,1].set_ylabel('Retorno')
        axes[1,1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
        
        # Adicionar valores nas barras
        for bar, return_val in zip(bars, returns):
            height = bar.get_height()
            axes[1,1].text(bar.get_x() + bar.get_width()/2., height,
                          f'{return_val:+.2%}',
                          ha='center', va='bottom' if height > 0 else 'top')
    
    plt.tight_layout()
    plt.show()

# Executar avaliação
if df_stock is not None and 'training_history' in locals():
    evaluation_results = evaluate_batman_agent(agent, env, num_test_episodes=20)
    plot_batman_results(training_history, evaluation_results)
else:
    print("⚠️ Execute primeiro o treinamento para avaliar o agente!")

## 🔧 TESTE COM OUTRO ATIVO

Para usar com outro ativo, simplesmente altere a variável `TICKER_SYMBOL` no início do notebook e execute novamente!

In [ ]:
# 🔧 Exemplo: Como trocar para outro ativo
"""
Para testar com VALE3, por exemplo:

1. Volte à segunda célula do notebook
2. Altere: TICKER_SYMBOL = "VALE3.SA"  
3. Execute novamente todas as células

O sistema automaticamente:
✅ Baixará os dados da VALE3
✅ Reconfigurará o ambiente
✅ Treinará o agente com os novos dados
✅ Avaliará a performance

Ativos suportados (B3):
- PETR3.SA, PETR4.SA (Petrobras)
- VALE3.SA (Vale)
- BRFS3.SA (BRF)
- ITUB4.SA (Itaú)
- BBAS3.SA (Banco do Brasil)
- ABEV3.SA (Ambev)
- E muitos outros...
"""

print("🔧 Sistema Batman configurado para flexibilidade!")
print(f"📊 Atualmente usando: {TICKER_SYMBOL}")
print("💡 Para trocar o ativo, altere TICKER_SYMBOL e re-execute o notebook!")

# Resumo final do Batman
print("\n" + "="*60)
print("🦇 RESUMO BATMAN Q-LEARNING")
print("="*60)
print("✅ Q-Learning clássico implementado")
print("✅ Estados discretizados para estabilidade") 
print("✅ Tabela Q tradicional")
print("✅ Exploração ε-greedy")
print("✅ Sistema flexível para qualquer ativo")
print("✅ Monitoramento completo de treinamento")
print("✅ Avaliação com benchmarks")
print("✅ Visualizações analíticas")
print("="*60)

# 🦇 BATMAN APPROACH - Reinforcement Learning Trading

## Estratégia: Metodológica e Estruturada

### Filosofia Batman
- **Preparação meticulosa**: Base teórica sólida seguindo padrões acadêmicos
- **Abordagem conservadora**: Q-Learning clássico com tabelas Q
- **Metodologia comprovada**: Seguindo estrutura similar às aulas do Prof. Paulo Caixeta
- **Estados discretizados**: Preços convertidos em faixas para simplicidade
- **Foco pedagógico**: Prioriza entendimento dos fundamentos de RL

### Objetivo
Desenvolver um agente de Reinforcement Learning para trading automatizado usando **Q-Learning tradicional**.
O sistema deve ser **genérico** e funcionar com qualquer ativo (PETR3, VALE3, BRFS3, etc.).

### Características da Implementação
- ✅ Q-Learning clássico com tabela Q
- ✅ Estados discretos (faixas de preços)
- ✅ Exploração ε-greedy
- ✅ Ambiente compatível com padrões Gymnasium
- ✅ Código flexível para múltiplos ativos
- ✅ Fundamentação teórica clara